# Text Processing 

In [2]:
import pandas as pd


In [3]:
df = pd.read_csv('./dataset/combined_news_excerpts.csv')
df

,Text
0,Starbucks violated federal labor law when it i...
1,The first suspect to plead guilty in Singapore...
2,Meta has been fined a record-breaking €1.2 bil...
3,SINGAPORE: A 45-year-old man linked to Singapo...
4,The Department of Education imposed a record $...
...,...
1519,METHODOLOGY\n\nThis investigation was conducte...
1520,"In February, 2002, DPKO Communications and Tec..."
1521,"""Another proposed topic was efforts by the Eur..."
1522,"""INTRODUCTION\n\nThis case arises out of an au..."


In [4]:
df1 = pd.read_excel('./dataset/wikileaks_parsed.xlsx')
df1

,PDF Path,Text
0,1.pdf,Pristina Airport – Possible administrative irr...
1,1.pdf,Investigative details\n\nIn his/her interviews...
2,10.pdf,"""An interoffice memorandum providing an “outst..."
3,10.pdf,"""Allegation 2 & 3:\n\n(Specifically, three of ..."
4,10.pdf,"""When asked about this in interview, the Divis..."
...,...,...
138,89.pdf,"""Description\n\nTop Secret US National Securit..."
139,9.pdf,"""INTRODUCTION\n\nThis case arises out of an au..."
140,9.pdf,"""BACKGROUND INFORMATION\n\nPristina Internatio..."
141,9.pdf,"""BACKGROUND INFORMATION\n\nPristina Internatio..."


In [5]:
import spacy

# Load spaCy
nlp = spacy.load("en_core_web_sm")

def get_keywords_spacy(text, top_n=5):
    """
    Tokenize using spaCy, remove stopwords/punctuation,
    lemmatize tokens, and then pick top_n frequent terms.
    """
    doc = nlp(text.lower().strip())
    
    # Filter out stopwords, punctuation, small tokens, etc.
    filtered_tokens = []
    for token in doc:
        if (
            not token.is_stop             # skip common stopwords
            and not token.is_punct        # skip punctuation
            and not token.like_num        # skip pure numbers
            and len(token) > 2            # skip tiny tokens like 'of', 'is'
        ):
            # Use the lemma form (e.g., "attacks" -> "attack")
            filtered_tokens.append(token.lemma_)
    
    # Count frequencies
    freq = {}
    for w in filtered_tokens:
        freq[w] = freq.get(w, 0) + 1
    
    # Sort by highest frequency
    sorted_words = sorted(freq.items(), key=lambda x: x[1], reverse=True)
    
    # Return top_n words as a set (for intersection usage)
    top_words = [word for (word, _) in sorted_words[:top_n]]
    return set(top_words)


In [6]:
import pandas as pd
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

#########################
# 1) Load Data
#########################
news_file = './dataset/combined_news_excerpts.csv'
wikileaks_file = './dataset/wikileaks_parsed.xlsx'

news_df = pd.read_csv(news_file)         # Should have columns: [Doc_ID, Text]
wiki_df = pd.read_excel(wikileaks_file)  # Should have columns: [Doc_ID, Text]

# If you don't have an existing Doc_ID, generate them:
# Example: N000, N001... and W000, W001...
if 'Doc_ID' not in news_df.columns:
    news_df['Doc_ID'] = ["N" + str(i).zfill(3) for i in news_df.index]
if 'Doc_ID' not in wiki_df.columns:
    wiki_df['Doc_ID'] = ["W" + str(i).zfill(3) for i in wiki_df.index]

#########################
# 2) Basic Preprocessing
#########################
def preprocess_text(text):
    if not isinstance(text, str):
        text = str(text)
    text = text.lower().strip()
    text = re.sub(r'\s+', ' ', text)      # remove excess whitespace
    return text

news_df['clean_text'] = news_df['Text'].apply(preprocess_text)
wiki_df['clean_text'] = wiki_df['Text'].apply(preprocess_text)

#########################
# 3) Load spaCy Model
#########################
nlp = spacy.load("en_core_web_sm")

#########################
# 4) Improved Keyword Extraction
#    - Remove stopwords, punctuation, short tokens
#    - Lemmatize tokens
#########################
def get_keywords_spacy(text, top_n=5):
    doc = nlp(text)
    tokens = []
    for token in doc:
        # Filter out stopwords, punctuation, numeric strings, etc.
        if (
            not token.is_stop
            and not token.is_punct
            and not token.like_num
            and len(token) > 2
        ):
            tokens.append(token.lemma_.lower())  # lemmatize
    # Frequency count
    freq = {}
    for w in tokens:
        freq[w] = freq.get(w, 0) + 1
    # sort by frequency desc
    sorted_words = sorted(freq.items(), key=lambda x: x[1], reverse=True)
    top_words = [word for word, _ in sorted_words[:top_n]]
    return set(top_words)

news_df['keywords'] = news_df['clean_text'].apply(get_keywords_spacy)
wiki_df['keywords'] = wiki_df['clean_text'].apply(get_keywords_spacy)

#########################
# 5) Named Entity Extraction
#    Filter to relevant entity labels
#########################
allowed_labels = {"ORG", "PERSON", "GPE", "LOC", "NORP"}

def extract_entities(text):
    doc = nlp(text)
    entities = set()
    for ent in doc.ents:
        if ent.label_ in allowed_labels:
            entities.add(ent.text.strip())
    return entities

news_df['entities'] = news_df['clean_text'].apply(extract_entities)
wiki_df['entities'] = wiki_df['clean_text'].apply(extract_entities)

#########################
# 6) TF-IDF Vector Calculation
#    Use ngram_range to capture some phrases
#########################
all_texts = news_df['clean_text'].tolist() + wiki_df['clean_text'].tolist()

# Example: use n-grams up to 2 to catch short phrases
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')
tfidf_matrix = vectorizer.fit_transform(all_texts)

num_news = len(news_df)
num_wiki = len(wiki_df)

# Split into two sub-matrices
news_vectors = tfidf_matrix[:num_news]
wiki_vectors = tfidf_matrix[num_news:]

#########################
# 7) Cosine Similarities
#########################
similarity_scores = cosine_similarity(news_vectors, wiki_vectors)

#########################
# 8) Build Final Similarity Table
#########################
rows = []

for i, news_row in news_df.iterrows():
    for j, wiki_row in wiki_df.iterrows():
        sim_score = similarity_scores[i, j]
        
        # Common Entities
        common_ents = news_row['entities'].intersection(wiki_row['entities'])
        common_ents_str = "; ".join(common_ents) if common_ents else "—"
        
        # Shared Keywords
        shared_kws = news_row['keywords'].intersection(wiki_row['keywords'])
        shared_kws_str = "; ".join(shared_kws) if shared_kws else "—"
        
        row_data = {
            "Doc_ID_News": news_row['Doc_ID'],
            "Doc_ID_Wiki": wiki_row['Doc_ID'],
            "Similarity_Score": round(sim_score, 2),
            "Common_Entities": common_ents_str,
            "Shared_Keywords": shared_kws_str
        }
        rows.append(row_data)

similarity_df = pd.DataFrame(rows)

#########################
# 9) Preview & Save
#########################
print(similarity_df.head(10))

# Save to CSV if needed
# similarity_df.to_csv("similarity_table.csv", index=False)



  Doc_ID_News Doc_ID_Wiki  Similarity_Score Common_Entities Shared_Keywords
0        N000        W000              0.01               —               —
1        N000        W001              0.02               —               —
2        N000        W002              0.00               —               —
3        N000        W003              0.00               —               —
4        N000        W004              0.00               —               —
5        N000        W005              0.00               —               —
6        N000        W006              0.01               —               —
7        N000        W007              0.00               —               —
8        N000        W008              0.01               —               —
9        N000        W009              0.00               —               —


In [7]:
similarity_df.sort_values('Similarity_Score', ascending=False, inplace=True)
similarity_df

,Doc_ID_News,Doc_ID_Wiki,Similarity_Score,Common_Entities,Shared_Keywords
217931,N1523,W142,1.0,the un oil­for­food (off); french; nzl; iraq; ...,report; french; company; u.s; plan
212949,N1489,W022,1.0,italian; europe; washington; east jerusalem; n...,italian; italy; help; israel; relation
215588,N1507,W087,1.0,unmik; kosovo; the kosovo force; united nation...,airport; include; investigation; own; pristina
216323,N1512,W107,1.0,kosovo; unmik; doti; united nations security c...,official; airport; doti; pristina; pillar
215002,N1503,W073,1.0,the kosovo trust agency; kosovo; london,fund; airport; insurance; coverage; manager
...,...,...,...,...,...
103646,N724,W114,0.0,—,—
103647,N724,W115,0.0,—,—
103648,N724,W116,0.0,—,—
103649,N724,W117,0.0,—,—


In [8]:
entity_df = pd.read_excel('./entities_dashboard_filtered.xlsx')
entity_df

,Dataset,Entity,Type,Start,End
0,0,Starbucks,ORG,0,9
1,0,National Labor Relations Board,ORG,127,157
2,0,NLRB,ORG,223,227
3,0,Starbucks,ORG,249,258
4,0,Starbucks,ORG,422,431
...,...,...,...,...,...
14596,1523,FTC,ORG,212,215
14597,1523,Avast,ORG,221,226
14598,1523,Avast,ORG,430,435
14599,1523,Czechoslovakia,GPE,513,527


In [9]:

# Suppose you have a DataFrame called similarity_df, with a column "Common_Entities"
# that contains values like: "the united states; kosovo; UNMIK"

# 1) Convert semicolon‐separated strings to lists
similarity_df["Common_Entities"] = (
    similarity_df["Common_Entities"]
    .fillna("")  # handle NaNs if any
    .apply(lambda x: [ent.strip() for ent in str(x).split(';') if ent.strip()])
)

# 2) Explode
# This creates multiple rows for each item in the list
similarity_exploded = similarity_df.explode("Common_Entities").reset_index(drop=True)

# Now each row has exactly ONE entity in "Common_Entities"
print(similarity_exploded.head(10))


  Doc_ID_News Doc_ID_Wiki  Similarity_Score            Common_Entities  \
0       N1523        W142               1.0  the un oil­for­food (off)   
1       N1523        W142               1.0                     french   
2       N1523        W142               1.0                        nzl   
3       N1523        W142               1.0                       iraq   
4       N1523        W142               1.0                 washington   
5       N1523        W142               1.0                     ts//si   
6       N1523        W142               1.0                       u.s.   
7       N1523        W142               1.0              david levitte   
8       N1523        W142               1.0                         un   
9       N1523        W142               1.0                     france   

                      Shared_Keywords  
0  report; french; company; u.s; plan  
1  report; french; company; u.s; plan  
2  report; french; company; u.s; plan  
3  report; french; compan

In [10]:
similarity_exploded

,Doc_ID_News,Doc_ID_Wiki,Similarity_Score,Common_Entities,Shared_Keywords
0,N1523,W142,1.0,the un oil­for­food (off),report; french; company; u.s; plan
1,N1523,W142,1.0,french,report; french; company; u.s; plan
2,N1523,W142,1.0,nzl,report; french; company; u.s; plan
3,N1523,W142,1.0,iraq,report; french; company; u.s; plan
4,N1523,W142,1.0,washington,report; french; company; u.s; plan
...,...,...,...,...,...
219163,N724,W114,0.0,—,—
219164,N724,W115,0.0,—,—
219165,N724,W116,0.0,—,—
219166,N724,W117,0.0,—,—


## Clean Entity Labels DF (Standardise)

In [11]:
entity_labels_df = pd.read_excel('./entities_dashboard_filtered.xlsx')
entity_labels_df['Entity']= entity_labels_df['Entity'].str.lower()

## Complete Merges to get Entity_Type for each Entity_Name

In [12]:
# Suppose entity_labels_df has columns: [Entity_Name, Entity_Type]
# Example: "the united states" -> GPE, "UNMIK" -> ORG

similarity_exploded = similarity_exploded.merge(
    entity_labels_df,
    left_on="Common_Entities",
    right_on="Entity",
    how="left")

# Drop the repeated "Entity_Name" column after merge
# similarity_exploded.drop(columns=["Entity"], inplace=True)

# Now you have columns like:
# [Doc_ID_News, Doc_ID_Wiki, Similarity_Score, Common_Entities, Entity_Type, Shared_Keywords, ...]


In [16]:
similarity_exploded.drop(columns=['Dataset', 'Start', 'End'], inplace=True)

In [18]:
similarity_exploded.rename(columns={
    'Type': 'Common_Entities_Type'
}, inplace=True)

In [22]:
similarity_exploded.to_csv('./dataset/tableau_chart1.csv')

In [123]:
import pandas as pd

# Load your data

# Exploding the 'Shared_Keywords' into separate rows
similarity_exploded['Shared_Keywords'] = similarity_exploded['Shared_Keywords'].str.split('; ')
exploded_df = similarity_exploded.explode('Shared_Keywords')




In [125]:
exploded_df.drop(columns=['category_label'], inplace=True)

In [127]:
exploded_df.to_csv('transformed_data.csv', index=False)